In [1]:
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
import numpy as np
from bokeh.plotting import figure, show
from bokeh.models import BoxSelectTool, Circle, CustomJS, ColumnDataSource
from bokeh.io import output_notebook
import pandas as pd

In [2]:
### BASIC DATA PROCESSING
fname = 'GephiMatrix_co-citation.csv'
f = open(fname, 'r')

# Get author names
line1 = f.readline()
names = line1[1:].split(';');

seen = {}
dupes = []

for index, name in enumerate(names):
    if name not in seen:
        seen[name] = 1
    else:
        if seen[name] == 1:
            dupes.append((index, name))
        seen[name] += 1

# add 1, 2 etc after the name
for pair in dupes:
    index = pair[0]
    name = pair[1]
    for i in range(seen[name]):
        names[index] = name + str((i+1))
        #print(names[index])

# Read csv
df = pd.read_csv(f, names=names, sep=';')

# Fix it
df = df.reset_index(level=1)
names.append("delete")
names = [name.replace('_', ' ') for name in names]
df.columns = names
del df["delete"]
df.set_index([df.columns], inplace=True)

# Get names again for later use
names = df.columns.tolist()

In [3]:
### BASIC DEGREE COUNTING

deg = []

N = len(names)
for name in names:
    deg.append([df[name][df[name] > 0].count()])

In [4]:

deg_plot = np.linspace(-max(deg)[0]/30, max(deg)+max(deg)[0]/30, 1000)


In [5]:
import time
start = time.time()
grid = GridSearchCV(KernelDensity(),
                    {'bandwidth': np.linspace(0.1, 10.0, 20)},
                    cv=5,
                    iid=False) # 5-fold cross-validation
grid.fit(deg)
print(grid.best_params_)
time.time()-start

{'bandwidth': 5.831578947368421}


4.903891324996948

In [6]:
kde = grid.best_estimator_
#kde = KernelDensity(kernel="gaussian", bandwidth=5.3).fit(deg)
log_dens = kde.score_samples(deg_plot)

In [7]:
X = np.append(deg_plot[:, 0], deg_plot[:, 0][-1])

In [8]:
Y = np.append(np.exp(log_dens), 0)

In [9]:
complete = ColumnDataSource(data=dict(x=X, y=Y))
before = ColumnDataSource(data=dict(x=[], y=[]))
middle = ColumnDataSource(data=dict(x=X, y=Y))
after = ColumnDataSource(data=dict(x=[], y=[]))

geometry_callback = CustomJS(args=dict(complete=complete, before=before, middle=middle, after=after, degrees=[item for sublist in deg for item in sublist]), code="""
let p = document.getElementById('between')
let geometry = cb_data["geometry"]

if(!p){
  p = document.createElement("p")
  p.id = "between"
  document.getElementsByClassName("bk-root")[0].appendChild(p)
}


let Xs = complete.data.x
let Ys = complete.data.y

let bXs = before.data.x
let bYs = before.data.y
bXs = []
bYs = []
let mXs = middle.data.x
let mYs = middle.data.y
mXs = []
mYs = []
let aXs = after.data.x
let aYs = after.data.y
aXs = []
aYs = []

for (let i = 0; i < Xs.length; i++){
  // should use binary search
  let x = Xs[i]
  let y = Ys[i]
  if(x < geometry.x0){
    bXs.push(x)
    bYs.push(y)
  }
  else if (x > geometry.x1){
    aXs.push(x)
    aYs.push(y)
  }
  else {
    mXs.push(x)
    mYs.push(y)
  }
}

bYs[0] = 0
bYs[bYs.length-1] = 0
mYs[0] = 0
mYs[mYs.length-1] = 0
aYs[0] = 0
aYs[aYs.length-1] = 0

before.data.x = bXs
before.data.y = bYs
middle.data.x = mXs
middle.data.y = mYs
after.data.x = aXs
after.data.y = aYs

before.change.emit()
middle.change.emit()
after.change.emit()

let amount = 0
for (let i = 0; i < degrees.length; i++){
  if (degrees[i] >= geometry.x0 && degrees[i] <= geometry.x1){
    amount++;
  }
}

colored_amount = "<span style='color:red'>" + amount + "</span>"
if (amount < 600){
  colored_amount = "<span style='color:orange'>" + amount + "</span>"
}
if (amount < 150){
  colored_amount = "<span style='color:green'>" + amount + "</span>"
}

p.innerHTML = "Selected " + colored_amount + " nodes with degree between " + Math.ceil(geometry.x0) + " and " + Math.floor(geometry.x1) + "."
""")

In [10]:
output_notebook()

p = figure(plot_width=700, plot_height=700)

select_tool = BoxSelectTool(dimensions="width", callback=geometry_callback)
p.add_tools(select_tool)

p.patch("x", "y", source=before, alpha=0.3, line_width=0)
p.patch("x", "y", source=middle, alpha=1, line_width=0, color="orange")
p.patch("x", "y", source=after, alpha=0.3, line_width=0)

p.xaxis.axis_label = "Degree"
p.yaxis.visible = False
p.grid.visible = False

p.toolbar.active_drag = select_tool
p.toolbar.autohide = True

show(p)

Loading BokehJS ...